# Sample Paper Trading Code File
#### Need to place MyDatabaseBreeze.py in the same folder. Change password in MyDatabaseBreeze.py file
#### Need to put sample data file tick.csv in the same folder

In [8]:
from MyDatabaseBreeze import DBHelper
import datetime
import warnings
warnings.filterwarnings('ignore')
import mysql.connector as sqlConnector
import pandas as pd
import time

### Fake Time Management Class to simulate Market timing

In [9]:
# Time management file for breeze simulation during paper trading on historical data
class Clock:
    def __init__(self, current_date):
        print('Fake Clock Initialized')
        self.current_datetime = datetime.datetime(current_date.year, current_date.month, current_date.day,9,15,0)
    
    def addOneSecond(self):
        self.current_datetime = self.current_datetime + datetime.timedelta(seconds=1)
        new_time = datetime.time(self.current_datetime.hour, self.current_datetime.minute, self.current_datetime.second)
        return new_time
    
    def addTenSecond(self):
        self.current_datetime = self.current_datetime + datetime.timedelta(seconds=10)
        new_time = datetime.time(self.current_datetime.hour, self.current_datetime.minute, self.current_datetime.second)
        return new_time
    
    def addOneMinute(self):
        self.current_datetime = self.current_datetime + datetime.timedelta(minutes=1)
        new_time = datetime.time(self.current_datetime.hour, self.current_datetime.minute, self.current_datetime.second)
        #print('new_time: ' + str(new_time))
        return new_time
    
    def addFiveMinute(self):
        self.current_datetime = self.current_datetime + datetime.timedelta(minutes=5)
        new_time = datetime.time(self.current_datetime.hour, self.current_datetime.minute, self.current_datetime.second)
        return new_time
    
    def addFifteenMinute(self):
        self.current_datetime = self.current_datetime + datetime.timedelta(minutes=15)
        new_time = datetime.time(self.current_datetime.hour, self.current_datetime.minute, self.current_datetime.second)
        return new_time
    
    def addOneDay(self):
        self.current_datetime = self.current_datetime + datetime.timedelta(days=1)
        self.current_datetime = datetime.datetime(self.current_datetime.year, self.current_datetime.month, self.current_datetime.day,9,16,10)
    
    def getCurrentDateTime(self):
        return self.current_datetime
    
    def getCurrentDate(self):
        cur_date = datetime.date(self.current_datetime.year, self.current_datetime.month, self.current_datetime.day)
        return cur_date
    
    def getCurrentTime(self):
        cur_time = datetime.time(self.current_datetime.hour, self.current_datetime.minute, self.current_datetime.second)
        return cur_time
    
    def setCurrentDateTime(self, current_date):
        self.current_datetime = datetime.datetime(current_date.year, current_date.month, current_date.day,9,16,10)
    
    def setCurrentDateTime2(self, current_date, hour, mins):
        self.current_datetime = datetime.datetime(current_date.year, current_date.month, current_date.day, hour, mins,0)

### Fake breeze API class
#### I am using zerodha name in database tables so that I can use it with Zerodha Also. Make approprite change as per your strategy
#### This is just for an Idea. Here we are getting sample data from csv file. Ideally you will fetch from database. Make appropriate changes

In [10]:
# Breeze API simulation
class BreezeFake:
    helper = DBHelper()
    starttime = datetime.time(9, 15, 0)
    global clock
    global DateRecords
    orderid = 0
    def __init__(self, current_date):
        print('Fake Breeze initialize method')
        self.orderid = 0
        self.current_date = current_date
        self.current_time = datetime.time(9, 15, 0)
        self.clock = Clock(current_date)
        
        #self.DateRecords = self.helper.GetTickData_fake_breeze('NIFTY-I.NFO', current_date, '09:15:00', '15:30:00')
        # Getting sample data from csv file. Ideally get it from database
        self.DateRecords = pd.read_csv('tick.csv')
        self.DateRecords["Time"]= self.DateRecords["Time"].astype(str)
        self.DateRecords['Time'] = self.DateRecords['Time'].str.slice(7, 15)
        self.DateRecords["Date"]= self.DateRecords["Date"].astype(str)
        self.DateRecords['Date'] = self.DateRecords['Date'] + ' ' +  self.DateRecords['Time']
        self.DateRecords['Date'] = pd.to_datetime(self.DateRecords['Date']) 
        
    def setCurrentDateTime2(self, date, hr, mn):
        self.clock.setCurrentDateTime2(date, hr, mn)
    def PlaceBuyOrderMarket(self, tradingsymbol, qty):
        dt = self.clock.getCurrentDateTime()
        self.helper.insert_fake_breeze_orders(tradingsymbol, qty, 'NFO', 'BUY', dt, 'MIS', 'Market', 0, 0)
        orderid = self.helper.get_fake_breeze_orderid()
        ltp = self.getCurrentLTP('NFO:' + tradingsymbol)
        self.helper.InsertTrade_Fake_breeze(orderid, 'NFO', tradingsymbol, ltp, qty, 'BUY', dt)
        self.helper.updateCompleteOrder_fake_breeze(orderid, ltp)    
        return str(orderid)

    def PlaceBuyOrderMarketNSE(self, tradingsymbol, qty):
        dt = self.clock.getCurrentDateTime()
        self.helper.insert_fake_breeze_orders(tradingsymbol, qty, 'NSE', 'BUY', dt, 'MIS', 'Market', 0, 0)
        orderid = self.helper.get_fake_breeze_orderid()
        ltp = self.getCurrentLTP('NSE:' + tradingsymbol)
        self.helper.InsertTrade_Fake_breeze(orderid, 'NSE', tradingsymbol, ltp, qty, 'BUY', dt)
        self.helper.updateCompleteOrder_fake_breeze(orderid, ltp)    
        return str(orderid)
    
    def PlaceBuyOrderLimit(self, tradingsymbol, qty, price):
        dt = self.clock.getCurrentDateTime()
        self.helper.insert_fake_breeze_orders(tradingsymbol, qty, 'NFO', 'BUY', dt, 'MIS', 'LIMIT', price, 0)
        orderid = self.helper.get_fake_breeze_orderid()
        return str(orderid)
    
    def PlaceSELLOrderMarket(self, tradingsymbol, qty):
        dt = self.clock.getCurrentDateTime()
        self.helper.insert_fake_breeze_orders(tradingsymbol, qty, 'NFO', 'SELL', dt, 'MIS', 'Market', 0, 0)
        orderid = self.helper.get_fake_breeze_orderid()
        ltp = self.getCurrentLTP('NFO:' + tradingsymbol)
        self.helper.InsertTrade_Fake_breeze(orderid, 'NFO', tradingsymbol, ltp, qty, 'SELL', dt)
        self.helper.updateCompleteOrder_fake_breeze(orderid, ltp)    
        return str(orderid)
    
    def PlaceSELLOrderMarketNSE(self, tradingsymbol, qty):
        dt = self.clock.getCurrentDateTime()
        self.helper.insert_fake_breeze_orders(tradingsymbol, qty, 'NSE', 'SELL', dt, 'MIS', 'Market', 0, 0)
        orderid = self.helper.get_fake_breeze_orderid()
        ltp = self.getCurrentLTP('NSE:' + tradingsymbol)
        self.helper.InsertTrade_Fake_breeze(orderid, 'NSE', tradingsymbol, ltp, qty, 'SELL', dt)     
        return str(orderid)
    
    def PlaceSELLOrderLimit(self, tradingsymbol, qty, price):
        dt = self.clock.getCurrentDateTime()
        self.helper.insert_fake_breeze_orders(tradingsymbol, qty, 'NFO', 'SELL', dt, 'MIS', 'LIMIT', price, 0)
        orderid = self.helper.get_fake_breeze_orderid()
        return str(orderid)
    
    def PlaceStopLossBuyOrderMarket(self, tradingsymbol, qty, triggerPrice):
        dt = self.clock.getCurrentDateTime()
        self.helper.insert_fake_breeze_orders(tradingsymbol, qty, 'NFO', 'BUY', dt, 'MIS', 'Market', 0, triggerPrice)
        orderid = self.helper.get_fake_breeze_orderid()
        return str(orderid)
    
    def PlaceStopLossBuyOrderMarketNSE(self, tradingsymbol, qty, triggerPrice):
        dt = self.clock.getCurrentDateTime()
        self.helper.insert_fake_breeze_orders(tradingsymbol, qty, 'NSE', 'BUY', dt, 'MIS', 'Market', 0, triggerPrice)
        orderid = self.helper.get_fake_breeze_orderid()
        return str(orderid)
    
    def PlaceStopLossSellOrderMarket(self, tradingsymbol, qty, triggerPrice):
        dt = self.clock.getCurrentDateTime()
        self.helper.insert_fake_breeze_orders(tradingsymbol, qty, 'NFO', 'SELL', dt, 'MIS', 'Market', 0, triggerPrice)
        orderid = self.helper.get_fake_breeze_orderid()
        return str(orderid)
    
    def PlaceStopLossSellOrderMarketNSE(self, tradingsymbol, qty, triggerPrice):
        dt = self.clock.getCurrentDateTime()
        self.helper.insert_fake_breeze_orders(tradingsymbol, qty, 'NSE', 'SELL', dt, 'MIS', 'Market', 0, triggerPrice)
        orderid = self.helper.get_fake_breeze_orderid()
        return str(orderid)
    
    def modifyStopLossOrderTriggerPrice(self, order_id, triggerPrice, parent_order):
        self.helper.update_SLtrigger_price_fake_breeze(triggerPrice, order_id)
        return order_id

    def gettrades(self):
        tm = self.clock.getCurrentDateTime()
        st = datetime.datetime(self.current_date.year, self.current_date.month, self.current_date.day, 0, 0, 1)
        et = datetime.datetime(self.current_date.year, self.current_date.month, self.current_date.day, 23, 59, 59)
        df = self.helper.GetTradeByDate_fake_breeze(st, et)
        dic = df.to_dict(orient='records')
        return dic
    
    def gettradeby_order(self, orderid):
        df = self.helper.GetTradeByOrderID_fake_breeze(orderid)
        dic = df.to_dict(orient='records')
        return dic
    
    def cancel_order(self, orderid):
        self.helper.update_order_status_fake_breeze('Cancelled', orderid)
        
    def getpositions(self):
        tm = self.clock.getCurrentDateTime()
        st = datetime.datetime(self.current_date.year, self.current_date.month, self.current_date.day, 0, 0, 1)
        et = datetime.datetime(self.current_date.year, self.current_date.month, self.current_date.day, 23, 59, 59)
        df = self.helper.GetPositions_fake_breeze()
        list1 = df.to_dict(orient='records')
        dic = {'net': list1}
        return dic
    
    def get_orders(self):
        tm = self.clock.getCurrentDateTime()
        st = datetime.datetime(self.current_date.year, self.current_date.month, self.current_date.day, 0, 0, 1)
        et = datetime.datetime(self.current_date.year, self.current_date.month, self.current_date.day, 23, 59, 59)
        df = self.helper.GetOrdersByDate_fake_breeze(st, et)
        dic = df.to_dict(orient='records')
        return dic

    def getOrderHistory(self, orderid):
        df = self.helper.getOrderHistory_fake_breeze(orderid)
        dic = df.to_dict(orient='records')
        return dic

    # def getOrderHistory(self, orderid):
    #     df = self.helper.GetTradeByOrderID_fake_breeze(orderid)
    #     dic = df.to_dict(orient='records')
    #     return dic        
    
    def gethistoricaldata(self, token, from_date, to_date, interval):
        ticker = 'NIFTY-I.NFO'
        stime = '09:15:00'
        etime = self.clock.getCurrentDateTime() - datetime.timedelta(minutes=1)
        new_time = datetime.time(etime.hour, etime.minute, 59)
        from_date = self.clock.getCurrentDate()
        if interval == 'second':
            etime = self.clock.getCurrentDateTime()
            new_time = datetime.time(etime.hour, etime.minute, etime.second)
        dataframe = self.DateRecords
        df1 = dataframe[dataframe['Date'] <= etime]
        df1["low"] = pd.to_numeric(df1["LTP"])
        df1["high"] = pd.to_numeric(df1["LTP"])
        df1["open"] = pd.to_numeric(df1["LTP"])
        df1["close"] = pd.to_numeric(df1["LTP"])
        df1["stdev"] = pd.to_numeric(df1["LTP"])
        
        df1.drop('Time', inplace=True, axis=1)   
        df1.drop('LTP', inplace=True, axis=1)
        df1.set_index('Date',inplace=True)
        dflow = df1.low.resample('1min').min()
        dfhigh = df1.high.resample('1min').max()
        dfopen = df1.open.resample('1min').first()
        dfstd = df1.stdev.resample('1min').std()
        dfclose = df1.close.resample('1min').last()
        
        dflow.dropna(axis = 0, how ='all', inplace = True)
        dfhigh.dropna(axis = 0, how ='all', inplace = True)
        dfopen.dropna(axis = 0, how ='all', inplace = True)
        dfclose.dropna(axis = 0, how ='all', inplace = True)
        dfstd.dropna(axis = 0, how ='all', inplace = True)
        
        df1 = pd.concat([dflow, dfhigh, dfopen, dfclose, dfstd], axis=1, join='inner')
        # df1["low"] = df1.low.shift(1)
        # df1["high"] = df1.high.shift(1)
        # df1["open"] = df1.open.shift(1)
        # df1["close"] = df1.close.shift(1)
        # df1.dropna(axis = 0, how ='all', inplace = True)
        df = df1.reset_index()
        return df
    def executeStopLossOrderSELL(self, orderid, tradingsymbol, qty):
        dt = self.clock.getCurrentDateTime()
        ltp = self.getCurrentLTP('NFO:' + tradingsymbol)
        self.helper.InsertTrade_Fake_breeze(orderid, 'NFO', tradingsymbol, ltp, qty, 'SELL', dt)
        self.helper.UpdateCurrentPhasebySymbol('unknown',tradingsymbol)
        self.helper.updateCompleteOrder_fake_breeze(orderid, ltp)
        self.helper.updateOrder('COMPLETE', ltp, orderid)
        df = self.helper.GetPositions_fake_breeze(tradingsymbol)
        if len(df) > 0:
            quantity = df.iloc[0, 3]
            newqty = quantity + (qty * -1)
            averagePrice = df.iloc[0, 4]
            if newqty != 0:
                newAvgPrice = ((averagePrice * abs(quantity)) + (ltp * qty)) / (abs(quantity) + qty)
                cost = newAvgPrice * newqty
                curval = ltp + newqty
                m2m = curval - cost
            else:
                newAvgPrice = 0
                m2m = 0
            self.helper.updatePosition_fake_breeze(newqty, newAvgPrice, ltp, m2m, tradingsymbol)        
    
    def executeStopLossOrderBUY(self, orderid, tradingsymbol, qty):
        dt = self.clock.getCurrentDateTime()
        ltp = self.getCurrentLTP('NFO:' + tradingsymbol)
        self.helper.InsertTrade_Fake_breeze(orderid, 'NFO', tradingsymbol, ltp, qty, 'BUY', dt)
        self.helper.UpdateCurrentPhasebySymbol('unknown',tradingsymbol)
        self.helper.updateCompleteOrder_fake_breeze(orderid, ltp)
        self.helper.updateOrder('COMPLETE', ltp, orderid)
        df = self.helper.GetPositions_fake_breeze(tradingsymbol)
        if len(df) > 0:
            quantity = df.iloc[0, 3]
            newqty = quantity + qty
            averagePrice = df.iloc[0, 4]
            if newqty != 0:
                newAvgPrice = ((averagePrice * abs(quantity)) + (ltp * qty)) / (abs(quantity) + qty)
                cost = newAvgPrice * newqty
                curval = ltp + newqty
                m2m = curval - cost
            else:
                newAvgPrice = 0
                m2m = 0
            self.helper.updatePosition_fake_breeze(newqty, newAvgPrice, ltp, m2m, tradingsymbol)        

    def getCurrentLTP(self, scripCode):
        ltp = 0
        df = self.helper.Get_NiftyFut_LTP_fake_breeze(self.clock.getCurrentDate(), self.clock.getCurrentTime())
        if len(df) > 0:
            ltp = df.iloc[0, 0]
            #print(str(ltp))
        else:
            print('Error getting LTP scrip' + scripCode + 'date: ' + str(self.clock.getCurrentDate()) + ' time: ' + str(self.clock.getCurrentTime()))
        return ltp
       
    def getLTPMulti(self, scrip_list):
        return '{}'
    
    def getCurrentDateTime(self):
            return self.clock.getCurrentDateTime()
        
    def getCurrentDate(self):
            return self.clock.getCurrentDate()
    
    def getCurrentTime(self):
        return self.clock.getCurrentTime()
    
    def addOneDay(self):
        self.clock.addOneDay()
        
    def setCurrentDate(self, current_date):
        self.current_date = current_date
        self.clock.setCurrentDateTime(current_date)
        
    def advance_time(self, duration):
        if duration == '1sec':
            self.current_time = self.clock.addOneSecond()
        elif duration == '10sec':
            self.current_time = self.clock.addTenSecond()
        elif duration == '1min':
            self.current_time = self.clock.addOneMinute()
    def Initiate(self):
        print('************* FAKE TRADE ***********')
        print('************* Breeze Simulation ***********')
        return True
    

target database host: localhost


### Get historical Data File
#### The code won't work on your machine since it requires market data stored in your database in Tick table

In [11]:
test_date = datetime.date(2021, 3, 1)
breeze = BreezeFake(test_date)
i = 0
while i < 10:
    df = breeze.gethistoricaldata('256265', test_date, test_date, 'second')
    df.set_index('Date', inplace=True)
    print(df[['open','high','low', 'close']].tail(1))
    #zeroda.clock.addTenSecond()
    breeze.clock.addOneMinute()
    i += 1
    time.sleep(1)


Fake Breeze initialize method
Fake Clock Initialized
                         open     high       low     close
Date                                                      
2021-03-01 09:15:00  14709.95  14743.3  14705.25  14705.25
                         open     high      low    close
Date                                                    
2021-03-01 09:15:00  14709.95  14743.3  14678.0  14678.0
                         open      high      low    close
Date                                                     
2021-03-01 09:17:00  14703.15  14703.15  14703.0  14703.0
                         open      high      low    close
Date                                                     
2021-03-01 09:17:00  14703.15  14705.25  14685.6  14698.0
                         open      high      low     close
Date                                                      
2021-03-01 09:18:00  14697.55  14702.95  14695.5  14700.45
                        open     high      low    close
Date              

In [12]:
breeze.PlaceBuyOrderMarket('NFO:NIFTY22AUGFUT', 50)

'1'

In [13]:
df = breeze.gethistoricaldata('256265','2021-03-01','2021-03-01','minute')
df

,Date,low,high,open,close,stdev
0,2021-03-01 09:15:00,14678.00,14743.30,14709.95,14678.00,10.518668
1,2021-03-01 09:16:00,14658.50,14702.50,14676.60,14702.45,14.375020
2,2021-03-01 09:17:00,14685.60,14705.25,14703.15,14698.00,4.636145
3,2021-03-01 09:18:00,14695.50,14702.95,14697.55,14700.45,1.381583
4,2021-03-01 09:19:00,14701.00,14723.00,14702.00,14723.00,4.764101
5,2021-03-01 09:20:00,14718.00,14749.00,14718.05,14739.75,7.548851
6,2021-03-01 09:21:00,14739.25,14754.00,14742.00,14743.00,3.465052
7,2021-03-01 09:22:00,14716.35,14750.00,14745.90,14716.35,7.942812
8,2021-03-01 09:23:00,14710.00,14732.05,14718.65,14727.75,6.263810
